Bibliotecas


In [1189]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import math
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity


Lendo dataframes


In [1190]:
dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])
filmesId=pd.DataFrame((set(dataOriginal['item id'])),columns=['Filmes'])

itemData=pd.read_csv('./ml-100k/u.item',sep=r'|',names=['movie_id ', 'movie title' , 'release date' ,' video release date' ,
              'IMDb URL' ,' unknown' , 'Action' , 'Adventure' , 'Animation' ,
             ' Childrens ', 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ], encoding='latin-1')
userData=pd.read_csv('./ml-100k/u.user',sep='|',names=['user id','age','Genero','Ocupacao','ZipCode'])

C:\Users\User\AppData\Local\Temp\ipykernel_8276\1420820392.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])


In [1191]:
def h(rat,user,mediaUser):
    val=rat - mediaUser[user-1]
    return val

In [1203]:
def cossine_similarity2(u, v,tabelaSimilaridade,id_u,id_v,w):
    itemsComuns = np.where((u!=0) & (v!=0))[0]
    if(itemsComuns.size==w):
        return 0
    u=u[itemsComuns]
    v=v[itemsComuns]
    if(u.size==0 or v.size==0):
        return 0
    result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))

    tabelaSimilaridade[id_u-1][id_v-1]=result
    tabelaSimilaridade[id_v-1][id_u-1]=result
    return result

In [1204]:
def user_based(user_id, item_id,data,tabelaSimilaridade,mediaUser,k,w,k_min):
        values_to_match = np.unique(data[:, item_id-1 ])
        values_to_match = values_to_match[values_to_match != 0]
        atching_indices = np.where(np.isin(data[:, item_id-1], values_to_match))[0]
        atching_indices+=1
        Nui = pd.DataFrame(userData[userData['user id'].isin(atching_indices)]['user id'].values,columns=['user id']).reset_index(drop=True)
        if(len(Nui)<=k_min):
            return mediaUser[user_id-1]
        Nui['similaridade']=Nui['user id'].apply(lambda x:(
                                  cossine_similarity2(data[user_id-1][:],
                                                    data[x-1][:],tabelaSimilaridade,user_id,x,w
                                                    )
                                                    if math.isnan(tabelaSimilaridade[user_id-1][x-1])else tabelaSimilaridade[user_id-1][x-1]))
        Nui=Nui[Nui['similaridade']!=0]
        Nui=Nui.sort_values(by='similaridade',ascending=False).head(k)
        parcial1=sum(Nui.apply(lambda x:
                                (x['similaridade']*h(data[int(x['user id'])-1][item_id-1],int(x['user id']),mediaUser)),axis=1
                                ))
        parcial2 = sum(Nui.apply(lambda x:
                                (abs(x['similaridade']) ),axis=1
                                ))
        val=mediaUser[user_id-1]
        resp=(parcial1/parcial2)+val
        # print(resp)    
        return resp

In [1205]:
userItem=dataOriginal.pivot(index='user id',columns='item id',values='rating').fillna(0)
userItem[:]=0
display(userItem)
userItem=userItem.to_numpy()
userItem

item id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [1206]:
# mediaUser=dataOriginal.drop(['timestamp','item id'],axis=1).groupby('user id').mean().reset_index()
# mediaUser=mediaUser.to_numpy()
# mediaUser

In [1207]:
#iniciando
similaridade = pd.DataFrame(index=userData['user id'],columns=userData['user id'])
for id in userData['user id']:
    similaridade[id][id]=1
display(similaridade)
similaridade=similaridade.to_numpy()
similaridade

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


array([[1, nan, nan, ..., nan, nan, nan],
       [nan, 1, nan, ..., nan, nan, nan],
       [nan, nan, 1, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., 1, nan, nan],
       [nan, nan, nan, ..., nan, 1, nan],
       [nan, nan, nan, ..., nan, nan, 1]], dtype=object)

In [1208]:
class UserBased():

    def __init__(self, tabelaSimilaridade,userItem,k=5,metrica='user_based',calculaSimilaridade='cosine',w=5,k_min=5):
        self.tabelaSimilaridade = tabelaSimilaridade
        self.userItem=userItem
        self.k=k
        self.metrica=metrica
        self.calculaSimilaridade=calculaSimilaridade
        self.w=w
        self.k_min=k_min

    def fit(self, x_train, y_train):
        x_train['rating']=y_train
        for index, row in x_train.iterrows():
            user_id=row['user id']
            item_id=row['item id']
            self.userItem[user_id-1][item_id-1]=row['rating']
        return self.userItem
    def predict(self, x):
        y_pred = []
        mediaUser=self.userItem.mean(axis=0)
        for index, row in x.iterrows():
            user_id=row['user id']
            item_id=row['item id']
            if(self.metrica=='user_based'):
                temp=user_based(user_id,item_id,self.userItem,self.tabelaSimilaridade,mediaUser,self.k,self.w,self.k_min)
                y_pred.append(temp)
                self.userItem[user_id-1][item_id-1]=temp
                mediaUser=self.userItem.mean(axis=0)
        return y_pred
        
    def evaluate(self, X, y):
        y_pred = self.predict(X)
        return mean_squared_error(y, y_pred)

In [1209]:
total=dataOriginal.drop(columns='timestamp')
total=total.sort_values(by='user id')
train,test=train_test_split(total,train_size=0.8)
x_train=train.drop('rating',axis=1)
y_train=train['rating']
x_test=test.drop('rating',axis=1)
y_test=test['rating']
x_test

,user id,item id
76468,765,50
9054,379,202
17663,383,9
84424,312,459
78066,185,258
...,...,...
25406,85,205
92712,538,50
71898,437,606
9206,269,506


In [1210]:
modelo=UserBased(similaridade,userItem,k=25,k_min=3)
modelo.fit(x_train,y_train)

array([[5., 3., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [1211]:
y_pred=modelo.predict(x_test)

In [1217]:
display(y_pred,y_test)


[3.725471898197243,
 3.699611059905567,
 3.610207960717237,
 3.4478660208943004,
 4.104814422057265,
 3.9560012970476537,
 3.396671830019269,
 3.7228505759459374,
 4.0991486404845645,
 4.30424317613967,
 4.068838415852849,
 3.378067905580452,
 3.4870242485135177,
 3.81210023098585,
 3.4658483183615387,
 3.535212694278942,
 2.69273776728435,
 4.426143535626478,
 4.022966962617393,
 4.885290922567698,
 4.395778823107711,
 3.3578400147238536,
 4.1235670397784885,
 3.706949673699281,
 4.180414701141242,
 3.8318066342212,
 4.183052384775864,
 2.807067812980057,
 3.853072958344492,
 2.9319808909562566,
 4.60441885645046,
 1.8778253585228741,
 2.54729826234421,
 4.785396606794121,
 4.031696957711987,
 4.004422386088721,
 3.904986941513619,
 2.5838301853000405,
 3.5363200244194326,
 3.38979962054112,
 4.211357269907729,
 3.1677561181946454,
 3.9510170824802913,
 4.793898786492269,
 3.837047079272158,
 1.5025963213073368,
 0.02014846235418876,
 3.949187282293127,
 4.6090152621701845,
 3.2716642

76468    2
9054     5
17663    5
84424    4
78066    4
        ..
25406    4
92712    5
71898    4
9206     5
31790    4
Name: rating, Length: 20000, dtype: int64

In [1215]:
mean_absolute_error(y_test,y_pred)

0.8730017031794816

In [1216]:
np.sqrt(mean_squared_error(y_test,y_pred))


1.126164975939606